In [ ]:
import torchvision
import os
import glob
import random
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import scipy.io as scp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import skimage.io as skio
import scipy.io as scp
from torch.utils.data import Dataset, DataLoader, Subset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class MyCNN(nn.Module):
  def __init__(self, num_channels, num_out_ch, img_w, img_h, num_classes):
    super(MyCNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=num_out_ch[0],
                           kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.bn1 = nn.BatchNorm2d(num_out_ch[0])
    self.conv2 = nn.Conv2d(in_channels=num_out_ch[0], out_channels=num_out_ch[1],
                           kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.bn2 = nn.BatchNorm2d(num_out_ch[1])
    self.conv3 = nn.Conv2d(in_channels=num_out_ch[1], out_channels=num_out_ch[1],
                           kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.bn3 = nn.BatchNorm2d(num_out_ch[1])
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.fc = nn.Linear(in_features = int(img_w//4)*int(img_h//4)*num_out_ch[1], out_features=num_classes)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.fc(x.reshape(x.shape[0], -1))

    return x


In [ ]:
NUM_OUT_CH = [8, 16]
IMAGE_W = 208
IMAGE_H = 208
BATCH_SIZE = 32
NUM_EPOCHS = 12
LR = 0.001

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# model
model = MyCNN(num_channels=3, num_out_ch=NUM_OUT_CH, img_w=IMAGE_W, img_h=IMAGE_H, num_classes=102)
model = model.to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr = LR)

# Loss Function
criterion = nn.CrossEntropyLoss()

In [ ]:
flower_transform = transforms.Compose([
    transforms.Resize((IMAGE_W, IMAGE_H)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

augmented_transform = transforms.Compose([
    transforms.Resize((IMAGE_W, IMAGE_H)),
    transforms.RandomHorizontalFlip(p=0.5),  # Flip the images randomly with a probability of 0.5
    transforms.RandomRotation(15),  # Randomly rotate images in the range (-15, 15) degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Randomly change brightness and contrast
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

root = '/content/drive/MyDrive/ActualFlowers/jpg'

train_set = torchvision.datasets.Flowers102(root = root, split = 'train', transform = augmented_transform, target_transform = None, download = False)
test_set = torchvision.datasets.Flowers102(root = root, split = 'test', transform = flower_transform, target_transform = None, download = False)
validation_set = torchvision.datasets.Flowers102(root = root, split = 'val', transform = flower_transform, target_transform = None, download = False)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
def check_accuracy(loader, model, num_classes=102):
    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    # Initialize the confusion matrix
    confusion_matrix = torch.zeros(num_classes, num_classes, dtype=torch.int64)

    with torch.no_grad():  # Do not calculate gradients
        for x, y in loader:
            x = x.to(device)  # Move data to the device
            y = y.to(device)  # Move labels to the device
            scores = model(x)  # Compute model output
            _, predictions = scores.max(1)  # Get the predicted classes
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

            # Update confusion matrix
            for t, p in zip(y.view(-1), predictions.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    model.train()  # Set the model back to training mode
    accuracy = float(num_correct) / num_samples  # Calculate accuracy

    # Print overall accuracy
    print(f"Got {num_correct} / {num_samples} with accuracy {accuracy * 100:.2f}%")

    # Print confusion matrix or other statistics if necessary
    # For detailed analysis, you might return or further process the confusion matrix
    return accuracy, confusion_matrix

In [ ]:
def evaluate(model, loader, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct_predictions / len(loader.dataset)
    model.train()  # Set the model back to training mode
    return avg_loss, accuracy

In [ ]:
best_val_loss = float('inf')
epochs_no_improve = 0
n_epochs_stop = 10
for epoch in range(NUM_EPOCHS*5):
    running_loss = 0
    with tqdm.tqdm(train_loader, unit='batch') as tepoch:
        for index, (x, y) in enumerate(tepoch):
            x = x.to(device)
            y = y.to(device)

            # Forward pass
            y_hat = model(x)
            loss = criterion(y_hat, y)
            running_loss += loss.item()

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item())

    # Compute average training loss
    avg_training_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}: Training loss: {running_loss:.4f}")
    check_accuracy(test_loader, model)

    # Evaluate on the validation set
    validation_loss, validation_accuracy = evaluate(model, validation_loader, device)
    print(f"Epoch {epoch}: Validation loss: {validation_loss:.4f}, Validation accuracy: {validation_accuracy*100:.4f}%")

    # Check for early stopping
    if validation_loss < best_val_loss:
        best_val_loss = validation_loss
        epochs_no_improve = 0
        # Save the model if validation loss improves
        torch.save(model.state_dict(), 'best_model_v1.pth')
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation loss for {epochs_no_improve} epochs.")

    # Early stopping condition
    if epochs_no_improve == n_epochs_stop:
        print("Early stopping triggered")
        break

100%|██████████| 32/32 [00:30<00:00,  1.06batch/s, loss=4.47]


Epoch 0: Training loss: 167.4602
Got 402 / 6149 with accuracy 6.54%
Epoch 0: Validation loss: 4.1430, Validation accuracy: 6.2745%


100%|██████████| 32/32 [00:30<00:00,  1.06batch/s, loss=3.68]


Epoch 1: Training loss: 111.1782
Got 818 / 6149 with accuracy 13.30%
Epoch 1: Validation loss: 3.6784, Validation accuracy: 17.6471%


100%|██████████| 32/32 [00:34<00:00,  1.07s/batch, loss=2.46]


...

Epoch 11: Training loss: 11.6304
Got 1425 / 6149 with accuracy 23.17%
Epoch 11: Validation loss: 4.7607, Validation accuracy: 27.5490%
No improvement in validation loss for 8 epochs.


100%|██████████| 32/32 [00:32<00:00,  1.03s/batch, loss=0.352]


Epoch 12: Training loss: 11.3056
Got 1470 / 6149 with accuracy 23.91%
Epoch 12: Validation loss: 4.6099, Validation accuracy: 28.2353%
No improvement in validation loss for 9 epochs.


100%|██████████| 32/32 [00:32<00:00,  1.02s/batch, loss=0.621]


Epoch 13: Training loss: 9.1760
Got 1564 / 6149 with accuracy 25.44%
Epoch 13: Validation loss: 4.5240, Validation accuracy: 30.5882%
No improvement in validation loss for 10 epochs.
Early stopping triggered
